In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

/usr/local/lib/python3.7/site-packages/python_jsonschema_objects/__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


In [2]:
d = {
  "sources": [
    [
      "91356497812394262",
      815458.9375,
      884707.6875,
      859720
    ]
  ],
  "sinks": [
    [
      "91356497812401228",
      816143.625,
      884547.3125,
      859720
    ]
  ]
}

In [4]:
cg.remove_edges(
    "test",
    source_ids=[91356497812394262],
    sink_ids=[91356497812401228],
    source_coords=[
        [
            815458.9375,
            884707.6875,
            859720
        ]
    ],
    sink_coords=[
        [
            816143.625,
            884547.3125,
            859720
        ]
    ],    
    
)

AttributeError: 'ChunkedGraph' object has no attribute 'get_unique_operation_id'